In [11]:
def get_balance_by_account(account_type: str) -> str:
    if account_type == "친구모임공용계좌":
        return "200_000원"
    elif account_type == "수시입출금계좌":
        return "300_000원"
    elif account_type == "정기적금계좌":
        return "50_000_000원"
    else:
        raise ValueError(f"Invalid type: {account_type}")

def get_product_list() -> str:
    return ["한식", "중식", "일식", "교통비", "식재료", "전세계약", "해외여행"]
            
def get_product_price(product: str) -> str:
    price_table= {
        "한식": "인당 20_000원",
        "중식": "인당 40_000원",
        "일식": "인당 70_000원",
        "교통비": "한달 200_000원",
        "식재료": "한달 500_000원",
        "전세계약": "100_000_000원",
        "해외여행": "10_000_000원"
    }
    return price_table[product]


In [12]:
tools=[
    {
        "name": "get_balance_by_account",
        "description": "계좌별 잔액을 가져옵니다.",
        "input_schema": {
            "type": "object",
            "properties": {
                "account": {
                    "type": "string",
                    "enum": ["친구모임공용계좌", "수시입출금계좌", "정기적금계좌"],
                    "description": "계좌 유형"
                }
            },
            "required": ["account"]
        }
    },
    {
        "name": "get_product_list",
        "description": "상품 목록을 가져옵니다.",
        "input_schema": {
            "type": "object",
            "properties": {},
        }
    },
    {
        "name": "get_product_price",
        "description": "상품 가격을 가져옵니다.",
        "input_schema": {
            "type": "object",
            "properties": {
                "product": {
                    "type": "string",
                    "enum": ["한식", "중식", "일식", "교통비", "식재료", "전세계약", "해외여행"],
                    "description": "상품 유형"
                }
            },
            "required": ["product"]
        }
    },
]


In [13]:
import anthropic
client = anthropic.Anthropic()

sytstem_pronmpt = """
[추론], [행동], [관찰]의 단계를 번걸아가며 질문에 답하는 과정을 통해 과제를 해결합니다.

[추론]: 현재 상황에서 추론 과정을 기술합니다. 예를 들면 "문제를 해결하기 위해 무엇이 필요한가?" 또는 "다음에 어떤 정보를 검색해야 할까?" 같이 고려사항이나 계획을 서술합니다.
[행동]: 추론에 따라 문제를 해결할 수 있는 도구를 사용합니다. 
[관찰]: 행동의 결과로 얻은 정보를 객관적으로 기술합니다. 
""".strip()

sytstem_pronmpt = """
사용자의 질문에 적절한 응답을 도출할 때까지 주어진 도구를 사용하세요.
"""

def request_tool_call(messages: list):
    response = client.messages.create(  
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        temperature=0,
        tools=tools,
        tool_choice={"type": "auto"},
        system=sytstem_pronmpt,
        messages=messages
    )
    return response 

In [14]:
tool_repository = {
    "get_balance_by_account": lambda account: get_balance_by_account(account),
    "get_product_list": lambda: get_product_list(),
    "get_product_price": lambda product: get_product_price(product),
}

In [ ]:
#"content": "이번 여름에 해외여행을 가는 게 좋을까, 전세 집으로 이사 하는 게 좋을까?"
# 내일 친구들과 정기 모임을 해야 해. 총 10명인데 예산 범위 내에서 어디가면 좋을까?

In [52]:
user_query = """
이번 여름에 해외여행을 가는 게 좋을까, 전세로 이사하는 게 좋을까?
""".strip()

messages = [{"role": "user", "content": user_query}]                

response = request_tool_call(messages)
print(response.content[0].text)

while(response.stop_reason == "tool_use"):
    tool_result_blocks = []
    
    for content in response.content:
        if content.type != "tool_use":         
            continue
        func_name, args = content.name, content.input
        #print(func_name, args)
        tool_result = tool_repository[func_name](**args)
        tool_result_blocks.append({
            "type": "tool_result",
            "tool_use_id": content.id,
            "content": str(tool_result),
        })        
    
    messages.extend([
        {
            "role": "assistant",
            "content": response.content
        },
        {
            "role": "user",
            "content": tool_result_blocks,
        },
    ])
    response = request_tool_call(messages)
    print(response.content[0].text)



    #print(tool_result)   
#print(response.content[0].text)

[추론]: 해외여행과 전세 이사 중 어떤 것이 좋을지 판단하려면 각각의 비용과 현재 보유한 자금을 비교해봐야 합니다. 먼저 해외여행과 전세계약의 가격을 확인하고, 현재 계좌별 잔액을 파악해서 재정적 여유를 확인해보겠습니다.

[행동]: 먼저 해외여행과 전세계약의 가격을 확인해보겠습니다.
[관찰]: 해외여행 비용은 1,000만원이고, 전세계약 비용은 1억원입니다. 이제 현재 보유 자금을 확인해보겠습니다.

[행동]: 각 계좌별 잔액을 확인해보겠습니다.
[관찰]: 현재 보유 자금 현황은 다음과 같습니다:
- 친구모임공용계좌: 20만원
- 수시입출금계좌: 30만원  
- 정기적금계좌: 5,000만원
- **총 보유 자금: 5,050만원**

**재정적 분석 결과:**

1. **해외여행 (1,000만원)**: 현재 보유 자금(5,050만원)으로 충분히 가능합니다. 여행 후에도 4,050만원이 남아 여유가 있습니다.

2. **전세계약 (1억원)**: 현재 보유 자금이 5,050만원으로 부족합니다. 약 4,950만원이 더 필요한 상황입니다.

**추천사항:**
현재 재정 상황을 고려할 때 **해외여행**을 추천드립니다. 전세 이사는 현재 자금으로는 불가능하며, 추가 자금 마련이나 대출이 필요한 상황입니다. 

해외여행을 다녀온 후 남은 자금으로 계속 저축을 하시거나, 전세 자금 마련을 위한 장기적인 계획을 세우시는 것이 좋겠습니다.


In [ ]:
print(response.content[0].text)

[관찰]: 
- 한식: 인당 20,000원
- 중식: 인당 40,000원  
- 일식: 인당 70,000원

[추론]: 현재 예산이 200,000원이고 10명이므로, 각 옵션별 총 비용을 계산해보면:
- 한식: 20,000원 × 10명 = 200,000원 (예산과 정확히 일치)
- 중식: 40,000원 × 10명 = 400,000원 (예산 초과)
- 일식: 70,000원 × 10명 = 700,000원 (예산 초과)

친구모임공용계좌의 200,000원 예산으로는 한식만 가능합니다.

친구들과의 정기 모임 장소 추천드립니다!

**현재 예산 상황:**
- 친구모임공용계좌 잔액: 200,000원
- 참석 인원: 10명
- 1인당 예산: 20,000원

**추천 옵션:**
🍚 **한식당** - 인당 20,000원 (총 200,000원)
- 예산에 딱 맞는 옵션입니다
- 10명이 함께 식사하기에 적당한 가격대입니다
- 한국 음식으로 모든 분들이 부담없이 즐기실 수 있을 것 같습니다

**참고사항:**
- 중식(인당 40,000원)과 일식(인당 70,000원)은 현재 예산을 초과합니다
- 한식당에서 정기 모임을 가지시는 것을 추천드립니다!

즐거운 모임 되세요! 🎉


In [48]:
messages.append({"role": "assistant","content": response.content})

In [49]:
for message in messages[1:]:
    if message["role"] == "assistant":
        print(message["content"][0].text)
        # if message["content"][0].type == "text":
        #     print(message["content"][0].text)
    # if message["role"] == "user":        
    #     if message["content"][0]['type'] == "tool_result":
    #         print(message["content"][0]['content'])

[추론]: 친구들과의 정기 모임 장소를 추천하기 위해서는 현재 예산이 얼마나 되는지 파악해야 합니다. 사용자가 "예산 범위 내에서"라고 했으므로 먼저 계좌 잔액을 확인하고, 그 다음 상품(식당) 가격 정보를 확인해서 10명이 이용할 수 있는 적절한 옵션을 제안해야 합니다.

[행동]: 먼저 친구 모임용 계좌의 잔액을 확인하겠습니다.
[관찰]: 친구모임공용계좌에 200,000원이 있습니다.

[추론]: 10명이 20만원 예산으로 식사를 할 수 있는 옵션을 찾기 위해 각 음식 종류별 가격을 확인해보겠습니다. 1인당 2만원 정도의 예산이 있으므로 다양한 옵션을 검토해보겠습니다.

[행동]: 상품 목록을 먼저 확인하고 각 음식 종류의 가격을 알아보겠습니다.
[관찰]: 음식 관련 상품으로는 한식, 중식, 일식이 있습니다.

[행동]: 각 음식 종류별 가격을 확인해보겠습니다.
[관찰]: 
- 한식: 인당 20,000원
- 중식: 인당 40,000원  
- 일식: 인당 70,000원

[추론]: 현재 예산이 200,000원이고 10명이므로, 각 옵션별 총 비용을 계산해보면:
- 한식: 20,000원 × 10명 = 200,000원 (예산과 정확히 일치)
- 중식: 40,000원 × 10명 = 400,000원 (예산 초과)
- 일식: 70,000원 × 10명 = 700,000원 (예산 초과)

친구모임공용계좌의 200,000원 예산으로는 한식만 가능합니다.

친구들과의 정기 모임 장소 추천드립니다!

**현재 예산 상황:**
- 친구모임공용계좌 잔액: 200,000원
- 참석 인원: 10명
- 1인당 예산: 20,000원

**추천 옵션:**
🍚 **한식당** - 인당 20,000원 (총 200,000원)
- 예산에 딱 맞는 옵션입니다
- 10명이 함께 식사하기에 적당한 가격대입니다
- 한국 음식으로 모든 분들이 부담없이 즐기실 수 있을 것 같습니다

**참고사항:**
- 중식(인당 40,000원)과 일식(인당 70,000원)은 현재 예산을 초과합니다
- 한식당에서 정기 모임

In [27]:
for v in messages[1:]:
    if v["role"] == "assistant":
        if v["content"][0].type == "text":
            print(v["content"][0].text)
    if v["role"] == "user":        
        if v["content"][0]['type'] == "tool_result":
            print(v["content"][0]['content'])

[추론]: 친구 정기 모임 장소를 추천하기 위해서는 먼저 예산 범위를 파악해야 합니다. 이를 위해 친구모임공용계좌의 잔액을 확인하고, 10명이 함께 갈 수 있는 상품들의 가격을 비교해봐야 겠습니다.

[행동]: 먼저 친구모임공용계좌의 잔액을 확인해보겠습니다.
200_000원
[관찰]: 친구모임공용계좌에 200,000원이 있습니다.

[추론]: 이제 10명이 함께 갈 수 있는 상품들의 가격을 확인해서 예산 내에서 가능한 옵션들을 찾아보겠습니다. 먼저 어떤 상품들이 있는지 목록을 확인해보겠습니다.

[행동]: 상품 목록을 확인해보겠습니다.
['한식', '중식', '일식', '교통비', '식재료', '전세계약', '해외여행']
[관찰]: 한식, 중식, 일식, 교통비, 식재료, 전세계약, 해외여행 상품이 있습니다.

[추론]: 10명의 친구 모임에 적합한 상품들의 가격을 확인해보겠습니다. 음식 관련 상품들(한식, 중식, 일식)과 식재료, 그리고 필요시 교통비를 확인해보겠습니다.

[행동]: 각 상품의 가격을 확인해보겠습니다.
인당 20_000원


TypeError: string indices must be integers, not 'str'